In [1]:
# Donchian Weekly Trend Following System
import gta_indicator
import gta_prices
import gta_signals
import numpy as np
import pandas as pd

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'GAMA'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
p1 = 48 # System 1 look back period.
p2 = 24 # System 2 look back period.
p3 = 12 # System 3 look back period.
p4 = 6  # System 4 look back period.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2 # Percentage risk per trade.
charges = 11.95  # Commission per trade.
sduty = 0.0  # Stamp Duty percentage.

In [3]:
# Function definitions.
def shares_div4(shares):
    '''Modify number of shares to be purchased to be divisible by 4.'''
    s = pd.Series(shares.values)
    for i in s.index:
        while s.iloc[i] % 4 != 0:
            s.iloc[i] += 1
    return shares

In [4]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm)

In [5]:
# Donchian channels.
dc1 = gta_indicator.donchian(prices, p1)
dc2 = gta_indicator.donchian(prices, p2)
dc3 = gta_indicator.donchian(prices, p3)
dc4 = gta_indicator.donchian(prices, p4)

In [6]:
# System 1 entries & exits.
s1 = pd.concat([prices, dc1], axis=1)
s1['sys'] = 1
s1['buy'] = np.where(s1.close > s1.upr, 1, 0)
s1['sell'] = np.where(s1.close < s1.mid, 1, 0)
s1['state'] = gta_signals.state_signal(s1.buy, s1.sell, p1)
s1['entry'] = np.where(np.logical_and(s1.state == 1, s1.state.shift(periods=1) == 0), 1, 0)
s1['exit'] = np.where(np.logical_and(s1.state == 0, s1.state.shift(periods=1) == 1), 1, 0)

In [7]:
# System 2 entries & exits.
s2 = pd.concat([prices, dc2], axis=1)
s2['sys'] = 2
s2['buy'] = s1.entry
s2['sell'] = np.where(s2.close < s2.mid, 1, 0)
s2['state'] = gta_signals.state_signal(s2.buy, s2.sell, p2)
s2['entry'] = np.where(np.logical_and(s2.state == 1, s2.state.shift(periods=1) == 0), 1, 0)
s2['exit'] = np.where(np.logical_and(s2.state == 0, s2.state.shift(periods=1) == 1), 1, 0)

In [8]:
# System 3 entries & exits.
s3 = pd.concat([prices, dc3], axis=1)
s3['sys'] = 3
s3['buy'] = s1.entry
s3['sell'] = np.where(s3.close < s3.mid, 1, 0)
s3['state'] = gta_signals.state_signal(s3.buy, s3.sell, p3)
s3['entry'] = np.where(np.logical_and(s3.state == 1, s3.state.shift(periods=1) == 0), 1, 0)
s3['exit'] = np.where(np.logical_and(s3.state == 0, s3.state.shift(periods=1) == 1), 1, 0)

In [9]:
# System 4 entries & exits.
s4 = pd.concat([prices, dc4], axis=1)
s4['sys'] = 4
s4['buy'] = s1.entry
s4['sell'] = np.where(s4.close < s4.mid, 1, 0)
s4['state'] = gta_signals.state_signal(s4.buy, s4.sell, p4)
s4['entry'] = np.where(np.logical_and(s4.state == 1, s4.state.shift(periods=1) == 0), 1, 0)
s4['exit'] = np.where(np.logical_and(s4.state == 0, s4.state.shift(periods=1) == 1), 1, 0)

In [10]:
# All entries & exits.
td = pd.concat([s1[s1.entry == 1] , s1[s1.exit == 1], s2[s2.exit == 1], s3[s3.exit == 1], s4[s4.exit == 1]], axis=0)
td = td.sort_index()

In [11]:
# Position size (buy).
td['volatility'] = np.where(td.entry == 1, abs((td.mid - td.close) / td.close).round(3), 0)
td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility).round(2), 0)
td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)

# Modify number of shares to be purchased to be divisible by 4.
td.shares = shares_div4(td.shares)

In [12]:
# Position size (sell).
for i in range(len(td)):
    if td.loc[td.index[i], 'exit'] == 0:
        shares = td.loc[td.index[i], 'shares']
    elif td.loc[td.index[i], 'exit'] == 1:
        td.loc[td.index[i], 'shares'] = shares / 4
td.shares = td.shares.astype('int')

In [13]:
# Charges & stamp duty.
td['charges'] = charges
td['sduty'] = ((sduty / 100) * td.close * td.shares).round(2)

# Cost (buy).
td['cost'] = np.where(td.entry == 1, (td.close * td.shares) + td.charges + td.sduty, 0)

# Cost (sell).
for i in range(len(td)):
    if td.loc[td.index[i], 'exit'] == 0:
        cost = td.loc[td.index[i], 'cost']
    elif td.loc[td.index[i], 'exit'] == 1:
        td.loc[td.index[i], 'cost'] = cost / 4
td.cost = td.cost.round(2)

# Value (sell).
td['value'] = np.where(td.exit == 1, ((td.close * td.shares) - td.charges).round(2), 0)

# Profit.
td['profit'] = np.where(td.exit == 1, td.value - td.cost, 0)

# Percentage return.
td['pct'] = np.where(td.exit == 1, ((td.profit / td.cost) * 100).round(1), 0)

# Trade duration.
for i in range(len(td)):
    if td.loc[td.index[i], 'entry'] == 1:
        start_date = td.index[i]
    elif td.loc[td.index[i], 'exit'] == 1:
        td.loc[td.index[i], 'days'] = td.index[i] - start_date
td.days = td.days.dt.days
td.days = td.days.fillna(0)
td.days = td.days.astype(int)

# Annual percentage return.
td['annual'] = ((np.power(1 + td.profit / td.cost, (365 / td.days)) - 1) * 100).round(1)

pd.set_option("display.max_columns", None)
td

,open,high,low,close,upr,lwr,mid,sys,buy,sell,state,entry,exit,volatility,risk_amt,shares,charges,sduty,cost,value,profit,pct,days,annual
date,,,,,,,,,,,,,,,,,,,,,,,,
2015-09-18,2.9400,3.2531,2.94,3.2163,2.97,2.0700,2.52000,1,1,0,1,1,0,0.216,6944.44,2160,11.95,0.0,6959.16,0.00,0.00,0.0,0,0.0
2016-01-01,4.3493,4.3950,4.30,4.3025,4.74,4.0375,4.38875,4,0,1,0,0,1,0.000,0.00,540,11.95,0.0,1739.79,2311.40,571.61,32.9,105,168.5
2016-01-22,4.3154,4.3975,3.88,4.1325,4.74,3.8200,4.28000,3,0,1,0,0,1,0.000,0.00,540,11.95,0.0,1739.79,2219.60,479.81,27.6,126,102.5
2016-04-01,4.1975,4.2700,4.13,4.2175,4.74,3.8200,4.28000,2,0,1,0,0,1,0.000,0.00,540,11.95,0.0,1739.79,2265.50,525.71,30.2,196,63.5
2018-11-16,8.2000,8.3200,7.06,7.3000,9.44,5.7000,7.57000,1,0,1,0,0,1,0.000,0.00,540,11.95,0.0,1739.79,3930.05,2190.26,125.9,1155,29.4
2019-03-15,8.7200,9.6200,8.58,9.5800,9.44,6.4000,7.92000,1,1,0,1,1,0,0.173,8670.52,908,11.95,0.0,8710.59,0.00,0.00,0.0,0,0.0
2019-06-07,12.5000,12.5000,10.55,10.7500,12.00,10.5000,11.25000,4,0,1,0,0,1,0.000,0.00,227,11.95,0.0,2177.65,2428.30,250.65,11.5,84,60.5
2019-07-05,11.2500,11.5500,10.50,10.8000,12.50,10.0000,11.25000,3,0,1,0,0,1,0.000,0.00,227,11.95,0.0,2177.65,2439.65,262.00,12.0,112,44.8
2019-07-19,10.3000,10.6000,9.38,10.1000,12.50,7.8000,10.15000,2,0,1,0,0,1,0.000,0.00,227,11.95,0.0,2177.65,2280.75,103.10,4.7,126,14.3


In [14]:
td.profit.sum()

4568.600000000002

In [15]:
td.index[-1] - td.index[0]

Timedelta('2191 days 00:00:00')

In [16]:
td.pct.sum()

246.39999999999998

In [17]:
len(td)

15